# 1. Generate Question

Let me explain the main input and output flows in `generateQuestion.js`:

### Main Input/Output Flow:

1. **Input (User Interface)**:
```javascript
// From standard form:
{
    topic: string,            // from #topic input
    level: string,           // from #level input
    questionCount: number,   // from #question-count input
    extraRequirements: string // from #extra-requirements input
}

// OR

// From custom form:
{
    customPrompt: string    // from #custom-prompt-text textarea
}
```

2. **API Call Output** (sent to backend):
```javascript
// POST to ${API_URL}/generate-questions
{
    prompt: string  // Either formatted standard prompt or custom prompt
}
```

3. **API Response Input** (received from backend):
```javascript
// Expected format:
[
    {
        question: string,
        structure: string,
        "main phrase": string,
        "optional phrase 1": string,
        "optional phrase 2": string,
        "question-vi": string,
        "structure-vi": string,
        "main phrase-vi": string,
        "optional phrase 1-vi": string,
        "optional phrase 2-vi": string
    },
    // ... more questions
]
```

4. **Final Output** (displayed to user):
- A table displaying all questions with their translations
- Each row has edit and delete functionality
- The data is stored in `storagedLessons` variable for further manipulation

### Key Processing Functions:

```javascript:frontend/src/js/generateQuestion.js
// Input processing
function createGenerateQuestionPrompt() {
    // Converts form inputs into formatted prompt string
}

// Output processing
function processApiResponse(data) {
    // Converts API response into usable lesson format
    // Adds unique lesson_id to each lesson
    // Stores in storagedLessons
}

function displayGeneratedQuestions(lessons) {
    // Renders lessons as HTML table
}
```

The flow is: User Input → API Request → API Response → Processed Data → Display Table


------------
@generateQuestion.js 

---------------------
Muốn tạo @tableDraftTracking.js , file này: 
---------------
1. mỗi lần ấn generatQuestion => sẽ tạo lesson_id  (topic_time_date, chẳng hạn: nametopic_2200_01112024   tạo lúc 22h ngày 1/11/2024)
2. input: là {
    topic: string,            // from #topic input
    level: string,           // from #level input
    questionCount: number,   // from #question-count input
    extraRequirements: string // from #extra-requirements input
}

3. raw 
sẽ là expected format 


    {
        question: string,
        structure: string,
        "main phrase": string,
        "optional phrase 1": string,
        "optional phrase 2": string,
        "question-vi": string,
        "structure-vi": string,
        "main phrase-vi": string,
        "optional phrase 1-vi": string,
        "optional phrase 2-vi": string
    },

4. final sẽ 
là JSON của bảng tạo ra (KHI NGƯỜI DÙNG ẤN Copy table) 
===============
File @tableDraftTracking.js này sẽ có trách nhiệm tạo 4 cái trên
sau đó call tới @tableDraftController.js để lưu data vào Larkbase



Tôi sẽ giúp bạn tạo `tableDraftTracking.js` để theo dõi và lưu trữ quá trình tạo câu hỏi:

```javascript:frontend/src/js/trackings/tableDraftTracking.js
import { config } from '../config.js';

const API_URL = config.production.apiUrl;

class TableDraftTracking {
    static generateLessonId(topic) {
        const now = new Date();
        const time = now.getHours().toString().padStart(2, '0') + 
                    now.getMinutes().toString().padStart(2, '0');
        const date = now.getDate().toString().padStart(2, '0') + 
                    (now.getMonth() + 1).toString().padStart(2, '0') + 
                    now.getFullYear();
        
        // Normalize topic name (remove spaces, special characters)
        const normalizedTopic = topic.toLowerCase()
            .replace(/[^a-z0-9]/g, '_')
            .replace(/_+/g, '_');
            
        return `${normalizedTopic}_${time}_${date}`;
    }

    static async trackDraftGeneration(inputData, rawResponse, finalTable = '') {
        try {
            const lesson_id = this.generateLessonId(inputData.topic);
            
            // Prepare input data
            const input = JSON.stringify({
                topic: inputData.topic,
                level: inputData.level,
                questionCount: inputData.questionCount,
                extraRequirements: inputData.extraRequirements
            });

            // Prepare raw data (API response)
            const raw = JSON.stringify(rawResponse);

            // Final data will be updated later when user clicks "Copy Table"
            const draftData = {
                lesson_id,
                input,
                raw,
                final: finalTable
            };

            const response = await fetch(`${API_URL}/submit-draft`, {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json'
                },
                body: JSON.stringify(draftData)
            });

            if (!response.ok) {
                throw new Error('Failed to submit draft tracking');
            }

            return lesson_id; // Return lesson_id for future reference
        } catch (error) {
            console.error('Error tracking draft:', error);
            throw error;
        }
    }

    static async updateFinalTable(lesson_id, finalTableData) {
        try {
            const draftData = {
                lesson_id,
                input: '',  // Không cần cập nhật lại
                raw: '',    // Không cần cập nhật lại
                final: JSON.stringify(finalTableData)
            };

            const response = await fetch(`${API_URL}/submit-draft`, {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json'
                },
                body: JSON.stringify(draftData)
            });

            if (!response.ok) {
                throw new Error('Failed to update final table data');
            }
        } catch (error) {
            console.error('Error updating final table:', error);
            throw error;
        }
    }
}

export default TableDraftTracking;
```

Và cập nhật `generateQuestion.js` để sử dụng tracking:

```javascript:frontend/src/js/generateQuestion.js
import TableDraftTracking from './trackings/tableDraftTracking.js';

// ... existing code ...

async function handleGenerateClick() {
    const activeTab = document.querySelector('.tab-content.active');
    const isStandardForm = activeTab.id === 'standard-form';
    
    if (isStandardForm) {
        const inputData = {
            topic: document.getElementById('topic').value,
            level: document.getElementById('level').value,
            questionCount: document.getElementById('question-count').value,
            extraRequirements: document.getElementById('extra-requirements').value
        };
        
        const prompt = createGenerateQuestionPrompt();
        const response = await generateQuestions(prompt);
        
        // Track the generation
        try {
            const lesson_id = await TableDraftTracking.trackDraftGeneration(
                inputData,
                response
            );
            // Store lesson_id for later use when copying table
            window.currentLessonId = lesson_id;
        } catch (error) {
            console.error('Failed to track draft generation:', error);
        }
    } else {
        // Handle custom prompt case...
    }
}

// Modify copyTableToClipboard function
async function copyTableToClipboard(table) {
    // ... existing copy logic ...

    // After successful copy, update the final table data
    if (window.currentLessonId) {
        try {
            const tableData = Array.from(table.querySelectorAll('tbody tr')).map(row => {
                const cells = Array.from(row.cells);
                return {
                    question: cells[0].textContent,
                    structure: cells[1].textContent,
                    'main phrase': cells[2].textContent,
                    'optional phrase 1': cells[3].textContent,
                    'optional phrase 2': cells[4].textContent,
                    'question-vi': cells[5].textContent,
                    'structure-vi': cells[6].textContent,
                    'main phrase-vi': cells[7].textContent,
                    'optional phrase 1-vi': cells[8].textContent,
                    'optional phrase 2-vi': cells[9].textContent
                };
            });

            await TableDraftTracking.updateFinalTable(window.currentLessonId, tableData);
        } catch (error) {
            console.error('Failed to update final table data:', error);
        }
    }
    
    alert('Table copied to clipboard!');
}
```

Chức năng chính:
1. Tạo `lesson_id` theo format yêu cầu
2. Lưu trữ input data từ form
3. Lưu trữ raw response từ API
4. Cập nhật final table data khi người dùng copy table
5. Gửi tất cả dữ liệu tới backend thông qua `submit-draft` endpoint

Khi sử dụng:
1. Mỗi lần generate questions sẽ tạo một bản ghi mới với `input` và `raw`
2. Khi user copy table, sẽ cập nhật `final` cho bản ghi đó
3. Tất cả dữ liệu được lưu vào Larkbase thông qua `tableDraftController.js`

dữ liệu mẫu sẽ gửi đến tableDraftController.js là gì ?




Tôi sẽ cho bạn ví dụ cụ thể về dữ liệu được gửi từ `tableDraftTracking.js` đến `tableDraftController.js`:

```javascript
// Example data sent to /submit-draft endpoint

{
    "lesson_id": "business_meeting_2200_01112024",
    
    "input": "{
        \"topic\": \"Business Meeting\",
        \"level\": \"Intermediate\",
        \"questionCount\": \"5\",
        \"extraRequirements\": \"Focus on professional vocabulary\"
    }",
    
    "raw": "[
        {
            \"question\": \"Which company are you working for?\",
            \"structure\": \"I'm the ____ from ABC Company.\",
            \"main phrase\": \"Sales representative\",
            \"optional phrase 1\": \"Sales director\",
            \"optional phrase 2\": \"Sales associate\",
            \"question-vi\": \"Bạn đang làm việc cho công ty nào vậy?\",
            \"structure-vi\": \"Tôi là ____ từ công ty ABC.\",
            \"main phrase-vi\": \"Đại diện kinh doanh\",
            \"optional phrase 1-vi\": \"Giám đốc kinh doanh\",
            \"optional phrase 2-vi\": \"Nhân viên bán hàng\"
        },
        // ... more questions
    ]",
    
    "final": "[
        {
            \"question\": \"Which company are you working for?\",
            \"structure\": \"I'm the ____ from ABC Company.\",
            \"main phrase\": \"Sales representative\",
            \"optional phrase 1\": \"Sales director\",
            \"optional phrase 2\": \"Sales associate\",
            \"question-vi\": \"Bạn đang làm việc cho công ty nào vậy?\",
            \"structure-vi\": \"Tôi là ____ từ công ty ABC.\",
            \"main phrase-vi\": \"Đại diện kinh doanh\",
            \"optional phrase 1-vi\": \"Giám đốc kinh doanh\",
            \"optional phrase 2-vi\": \"Nhân viên bán hàng\"
        },
        // ... edited/final questions after user modifications
    ]"
}
```

Giải thích từng trường:

1. **lesson_id**: 
   - Format: `topic_time_date`
   - Example: `business_meeting_2200_01112024`

2. **input**: JSON string chứa thông tin form nhập
   ```json
   {
       "topic": "Business Meeting",
       "level": "Intermediate",
       "questionCount": "5",
       "extraRequirements": "Focus on professional vocabulary"
   }
   ```

3. **raw**: JSON string chứa response gốc từ API
   ```json
   [
       {
           "question": "Which company are you working for?",
           "structure": "I'm the ____ from ABC Company.",
           "main phrase": "Sales representative",
           // ... other fields
       },
       // ... more questions
   ]
   ```

4. **final**: JSON string chứa dữ liệu cuối cùng sau khi user edit và copy
   ```json
   [
       {
           "question": "Which company are you working for?",
           "structure": "I'm the ____ from ABC Company.",
           "main phrase": "Sales representative",
           // ... other fields
       },
       // ... final edited questions
   ]
   ```

Lưu ý:
- Tất cả các trường object/array đều được chuyển thành JSON string
- `final` có thể khác với `raw` nếu user đã chỉnh sửa câu hỏi
- Dữ liệu được gửi trong 2 thời điểm:
  1. Khi generate questions (`input` và `raw`)
  2. Khi copy table (cập nhật `final`)



Lỗi này xảy ra do vấn đề với JSON string. Hãy sửa lại cách xử lý dữ liệu trong `tableDraftTracking.js`:

```javascript:frontend/src/js/trackings/tableDraftTracking.js
import { config } from '../config.js';

const API_URL = config.production.apiUrl;

class TableDraftTracking {
    static generateLessonId(topic) {
        const now = new Date();
        const time = now.getHours().toString().padStart(2, '0') + 
                    now.getMinutes().toString().padStart(2, '0');
        const date = now.getDate().toString().padStart(2, '0') + 
                    (now.getMonth() + 1).toString().padStart(2, '0') + 
                    now.getFullYear();
        
        const normalizedTopic = topic.toLowerCase()
            .replace(/[^a-z0-9]/g, '_')
            .replace(/_+/g, '_');
            
        return `${normalizedTopic}_${time}_${date}`;
    }

    static async trackDraftGeneration(inputData, rawResponse, finalTable = '') {
        try {
            const lesson_id = this.generateLessonId(inputData.topic);
            
            // Clean and prepare data
            const cleanInput = {
                topic: inputData.topic || '',
                level: inputData.level || '',
                questionCount: inputData.questionCount || '',
                extraRequirements: inputData.extraRequirements || ''
            };

            // Ensure data is properly stringified
            const draftData = {
                lesson_id: lesson_id,
                input: JSON.stringify(cleanInput),
                raw: JSON.stringify(rawResponse),
                final: finalTable ? JSON.stringify(finalTable) : ''
            };

            const response = await fetch(`${API_URL}/submit-draft`, {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json'
                },
                body: JSON.stringify(draftData)
            });

            if (!response.ok) {
                throw new Error('Failed to submit draft tracking');
            }

            return lesson_id;
        } catch (error) {
            console.error('Error tracking draft:', error);
            throw error;
        }
    }

    static async updateFinalTable(lesson_id, finalTableData) {
        try {
            // Clean the table data before sending
            const cleanTableData = finalTableData.map(row => ({
                question: row.question || '',
                structure: row.structure || '',
                'main phrase': row['main phrase'] || '',
                'optional phrase 1': row['optional phrase 1'] || '',
                'optional phrase 2': row['optional phrase 2'] || '',
                'question-vi': row['question-vi'] || '',
                'structure-vi': row['structure-vi'] || '',
                'main phrase-vi': row['main phrase-vi'] || '',
                'optional phrase 1-vi': row['optional phrase 1-vi'] || '',
                'optional phrase 2-vi': row['optional phrase 2-vi'] || ''
            }));

            const draftData = {
                lesson_id,
                input: '',  // Không cần cập nhật
                raw: '',    // Không cần cập nhật
                final: JSON.stringify(cleanTableData)
            };

            const response = await fetch(`${API_URL}/submit-draft`, {
                method: 'POST',
                headers: {
                    'Content-Type': 'application/json'
                },
                body: JSON.stringify(draftData)
            });

            if (!response.ok) {
                throw new Error('Failed to update final table data');
            }
        } catch (error) {
            console.error('Error updating final table:', error);
            throw error;
        }
    }
}

export default TableDraftTracking;
```

Những thay đổi chính:
1. Thêm clean data trước khi stringify
2. Xử lý các trường hợp undefined/null
3. Đảm bảo tất cả dữ liệu được stringify đúng cách
4. Thêm kiểm tra và xử lý lỗi chi tiết hơn

Dữ liệu gửi đi sẽ có dạng:
```javascript
{
    "lesson_id": "business_meeting_2200_01112024",
    "input": "{\"topic\":\"Business Meeting\",\"level\":\"Intermediate\",\"questionCount\":\"5\",\"extraRequirements\":\"Focus on professional vocabulary\"}",
    "raw": "[{\"question\":\"Which company...\"}]",
    "final": "[{\"question\":\"Which company...\"}]"
}
```

Lưu ý:
- Tất cả các string JSON phải được escape đúng cách
- Không được có ký tự control trong string
- Đảm bảo dữ liệu được làm sạch trước khi stringify